In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [2]:
BATCH_SIZE = 100
DATASETSIZE = 800
LEARNING_RATE_BASE = 0.99
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.08#正则化率
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99
PURE_LOSS_WIGHT = 0.5
L1_OVER_L1ADDL2 = 0.5
# 模型保存的路径和文件名
MODEL_SAVE_PATH = "TTNKmodels"
MODEL_NAME = "model.ckpt"

# 定义神经网路相关参数|
INPUT_NODE = 33
OUTPUT_NODE = 2
LAYER1_NODE = 600

In [3]:
folder = 'TTNKdata'
_train_path=os.path.join(folder,'prepressed_train_data.csv')
train_data=pd.read_csv(_train_path,header = None)

In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1.0,3.0,1.0,22.0,1.0,0.0,7.2500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,1.0,0.0,38.0,1.0,0.0,71.2833,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,1.0
2,3.0,3.0,0.0,26.0,0.0,0.0,7.9250,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,4.0,1.0,0.0,35.0,1.0,0.0,53.1000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0,1.0
4,5.0,3.0,1.0,35.0,0.0,0.0,8.0500,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
np.array(train_data).shape

(891, 35)

In [6]:
#X Y 分别为属性和标签
train_Y = np.array(train_data)[:DATASETSIZE,34]
Y = []
for y in train_Y:
    if y==0:
        Y.append([1,0])
    else:
        Y.append([0,1])
print(Y)
X = np.array(train_data)[:DATASETSIZE,1:34]
X = X.astype(np.float32)
    #numpy默认是float64，跟神经网络定义的时候的变量类型(float32)不符合，会出点问题
    #使用astype()改变类型
print(X.shape)
print(X[1])

[[1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0],

In [ ]:
test_data.head()

In [8]:
#构造测试集
val_y = np.array(train_data)[DATASETSIZE:,34]
val_Y = []
for y in val_y:
    if y==0:
        val_Y.append([1,0])
    else:
        val_Y.append([0,1])
val_Y = np.array(val_Y)
print(val_Y)
val_X = np.array(train_data)[DATASETSIZE:,1:34]
val_X = val_X.astype(np.float32)
    #numpy默认是float64，跟神经网络定义的时候的变量类型(float32)不符合，会出点问题
    #使用astype()改变类型
print(val_X.shape)
print(type(X))
print(type(val_X[0][1]))

[[1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]]
(91, 33)
<class 'numpy.ndarray'>
<class 'numpy.float32'>


In [9]:
#获取变量
def get_weights_variable(shape, regularizer):
    weights = tf.get_variable(
        "weights", shape,dtype=tf.float32,
        initializer=tf.truncated_normal_initializer(stddev=0.1))

    # 当给出了正则化函数时，将当前变量的正则化损失加入名字为losses的集合当中
    # 这个集合不再TF自动管理的集合列表中
    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

In [10]:
# 定义神经网路的向前传播过程
def inference(input_tensor, regularizer):  # regulaizer 正则化
    # 声明第一层神经网路的变量并完成向前传播过程
    with tf.variable_scope("layer1",reuse=tf.AUTO_REUSE):
        #  如果存在多次调用该函数，需要在第一次调用后将reuse设置为True
        weights = get_weights_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases) #Acc on val is 0.857143
        #layer1 = tf.nn.leaky_relu(tf.matmul(input_tensor, weights) + biases)#Acc on val is 0.868132
        
        # 第二层神经网络
    with tf.variable_scope("layer2",reuse = tf.AUTO_REUSE):
        weights = get_weights_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    #使用sotfmax转化为概率值
    layer_to_return = tf.nn.softmax(layer2)
    # 返回最后的向前传播结果
    return layer_to_return


In [11]:
def acc(val_X,val_Y,sess):
    #numpy默认是float64，跟神经网络定义的时候的变量类型(float32)不符合，会出点问题
    #使用astype()改变类型
    #print(type(val_X)) #<class 'numpy.ndarray'>
    local_Y = sess.run(inference(val_X,regularizer=None))
    right = 0.0
    counts = len(val_Y)
    for local_y,val_y in zip(local_Y,val_Y):
        if local_y[0]>=local_y[1] and val_y[0]==1:
            right+=1
        elif local_y[0]<local_y[1] and val_y[0]==0:
            right+=1
        else:
            pass
    return right/counts
    
#     correct_prediction = tf.equal(tf.argmax(val_Y, 1), tf.argmax(local_Y, 1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  # casts a tensor to a new type
    
#     return sess.run(accuracy)

In [52]:
def train(train_data):
    X, Y = train_data[0],train_data[1]
    # 定义输入输出placeholder
    x = tf.placeholder(tf.float32, [None,INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None,OUTPUT_NODE], name="y-input")
    regularizer = tf.contrib.layers.l1_l2_regularizer(REGULARAZTION_RATE*L1_OVER_L1ADDL2,REGULARAZTION_RATE*(1-L1_OVER_L1ADDL2))#希望矩阵更稀疏
#     Args:
#       scale_l1: A scalar multiplier `Tensor` for L1 regularization.
#       scale_l2: A scalar multiplier `Tensor` for L2 regularization.
#       scope: An optional scope name.

    # 直接使用_inference中的向前传播过程
    y = inference(x, regularizer)
    global_steps = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作及训练过程
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_steps)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    try:
        variable_averages_op = variable_averages.apply(tf.trainable_variables())
    except:
        pass
    
    #不均衡的数据集合，使用自定义损失
    #pure_loss = tf.reduce_sum()
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_sum = tf.reduce_sum(cross_entropy)#改成reducesum后好了很多
    loss = cross_entropy_sum*PURE_LOSS_WIGHT + tf.add_n(tf.get_collection('losses'))*(1-PURE_LOSS_WIGHT)
    
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_steps,
        len(X) / BATCH_SIZE*10,
        LEARNING_RATE_DECAY)
    
    train_step = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_steps)
    #train_step = tf.train.GradientDescentOptimizer(learning_rate) \
       # .minimize(loss, global_step=global_steps)#更换优化器
    # 下面是一次性完成两个op的操作
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化TF持久化类
    saver = tf.train.Saver()
    
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        train_writer = tf.summary.FileWriter('TTNKlogs/',sess.graph)#定义一个写入summary的目标文件，dir为写入文件地址  
        tf.summary.scalar('loss',loss) #loss标量的图
        merge_summary = tf.summary.merge_all()
        #merge_summary = tf.summary.merge([tf.get_collection(tf.GraphKeys.SUMMARIES,'losses')])  #使用tf.get_collection函数筛选图中summary信息中的accuracy信
        # 这里只有训练过程，没有验证和测试过程
        for i in range(TRAINING_STEPS):
            start = (i*BATCH_SIZE)%DATASETSIZE
            end = min(start+BATCH_SIZE,DATASETSIZE)
            _, loss_value, step = sess.run([train_op, loss, global_steps], feed_dict={x:X[start:end],y_:Y[start:end]})
            # 每1000轮保存一次模型
            if True:
                train_summary = sess.run(merge_summary,feed_dict={x:X[start:end],y_:Y[start:end]})#调用sess.run运行图，生成一步的训练过程数据  
                train_writer.add_summary(train_summary,step)#调用train_writer的add_summary方法将训练过程以及训练步数保存  
            if i % 1000 == 0:
                # 输出当前轮的训练情况，这里是输出当前batch的损失函数的大小。
                # 正确率会由一个单独的程序来生成
                print ("After %d training step(s), loss on training "
                       "batch is %g." % (step, loss_value))
                pure_loss = sess.run(cross_entropy_sum,feed_dict={x:X[start:end],y_:Y[start:end]})
                print("After %d training step(s), pure_loss on training "
                       "batch is %g." % (step, pure_loss))
                print("After %d training step(s), regulayzer on training "
                       "batch is %g." % (step,loss_value - pure_loss*PURE_LOSS_WIGHT))
                # 保存当前的模型。并在模型名后添加训练的轮数
                print("Acc on val is %f"%acc(val_X,val_Y,sess))
                print("Acc on test is %f"%acc(X,Y,sess))
                saver.save(
                    sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME),
                    global_step=global_steps)
#             if i % 5000 ==0:
#                 print(sess.run(inference(X,regularizer=None)))

In [55]:
# 初始化计算图
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.999
REGULARAZTION_RATE = 0.07#正则化率
TRAINING_STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
PURE_LOSS_WIGHT = 0.5
L1_OVER_L1ADDL2 = 0.6

try:
    tf.reset_default_graph()
except:
    pass
train([X,Y])

After 1 training step(s), loss on training batch is 78.458.
After 1 training step(s), pure_loss on training batch is 90.3212.
After 1 training step(s), regulayzer on training batch is 33.2974.
Acc on val is 0.373626
Acc on test is 0.385000
After 1001 training step(s), loss on training batch is 42.006.
After 1001 training step(s), pure_loss on training batch is 52.4927.
After 1001 training step(s), regulayzer on training batch is 15.7596.
Acc on val is 0.813187
Acc on test is 0.816250
After 2001 training step(s), loss on training batch is 33.382.
After 2001 training step(s), pure_loss on training batch is 52.5882.
After 2001 training step(s), regulayzer on training batch is 7.08794.
Acc on val is 0.813187
Acc on test is 0.820000
After 3001 training step(s), loss on training batch is 32.0056.
After 3001 training step(s), pure_loss on training batch is 52.6772.
After 3001 training step(s), regulayzer on training batch is 5.66705.
Acc on val is 0.835165
Acc on test is 0.825000
After 4001 t

KeyboardInterrupt: 

In [39]:
LEARNING_RATE_BASE = 0.9
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.07#正则化率
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99
PURE_LOSS_WIGHT = 0.5
L1_OVER_L1ADDL2 = 0.3
# Acc on val is 0.879121
# Acc on test is 0.846250

In [ ]:
# 初始化计算图
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.999
REGULARAZTION_RATE = 0.07#正则化率
TRAINING_STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
PURE_LOSS_WIGHT = 0.5
L1_OVER_L1ADDL2 = 0.6
# Acc on val is 0.868132
# Acc on test is 0.872500

In [56]:
#选定参数后使用全部数据训练：
train_Y = np.array(train_data)[:,34]
Y = []
for y in train_Y:
    if y==0:
        Y.append([1,0])
    else:
        Y.append([0,1])
print(Y)
X = np.array(train_data)[:,1:34]
X = X.astype(np.float32)
    #numpy默认是float64，跟神经网络定义的时候的变量类型(float32)不符合，会出点问题
    #使用astype()改变类型
print(X.shape)
print(X[1])

[[1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0],

In [57]:
DATASETSIZE = 891

In [59]:
# 初始化计算图
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.999
REGULARAZTION_RATE = 0.07#正则化率
TRAINING_STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
PURE_LOSS_WIGHT = 0.5
L1_OVER_L1ADDL2 = 0.6

try:
    tf.reset_default_graph()
except:
    pass
train([X,Y])

After 1 training step(s), loss on training batch is 68.4857.
After 1 training step(s), pure_loss on training batch is 70.4854.
After 1 training step(s), regulayzer on training batch is 33.2431.
Acc on val is 0.626374
Acc on test is 0.616162
After 1001 training step(s), loss on training batch is 42.3403.
After 1001 training step(s), pure_loss on training batch is 51.9782.
After 1001 training step(s), regulayzer on training batch is 16.3512.
Acc on val is 0.868132
Acc on test is 0.824916
After 2001 training step(s), loss on training batch is 33.9729.
After 2001 training step(s), pure_loss on training batch is 52.1765.
After 2001 training step(s), regulayzer on training batch is 7.88469.
Acc on val is 0.879121
Acc on test is 0.831650
After 3001 training step(s), loss on training batch is 29.5565.
After 3001 training step(s), pure_loss on training batch is 46.386.
After 3001 training step(s), regulayzer on training batch is 6.36352.
Acc on val is 0.879121
Acc on test is 0.826038
After 4001

After 33001 training step(s), loss on training batch is 26.1603.
After 33001 training step(s), pure_loss on training batch is 44.8759.
After 33001 training step(s), regulayzer on training batch is 3.72237.
Acc on val is 0.890110
Acc on test is 0.860831
After 34001 training step(s), loss on training batch is 15.5282.
After 34001 training step(s), pure_loss on training batch is 23.7909.
After 34001 training step(s), regulayzer on training batch is 3.63278.
Acc on val is 0.890110
Acc on test is 0.859708
After 35001 training step(s), loss on training batch is 27.0072.
After 35001 training step(s), pure_loss on training batch is 46.8508.
After 35001 training step(s), regulayzer on training batch is 3.5818.
Acc on val is 0.879121
Acc on test is 0.859708
After 36001 training step(s), loss on training batch is 27.3488.
After 36001 training step(s), pure_loss on training batch is 47.5225.
After 36001 training step(s), regulayzer on training batch is 3.58754.
Acc on val is 0.890110
Acc on test i

After 66001 training step(s), loss on training batch is 26.6356.
After 66001 training step(s), pure_loss on training batch is 46.9777.
After 66001 training step(s), regulayzer on training batch is 3.14676.
Acc on val is 0.879121
Acc on test is 0.869809
After 67001 training step(s), loss on training batch is 25.4592.
After 67001 training step(s), pure_loss on training batch is 44.4393.
After 67001 training step(s), regulayzer on training batch is 3.23954.
Acc on val is 0.890110
Acc on test is 0.865320
After 68001 training step(s), loss on training batch is 24.2014.
After 68001 training step(s), pure_loss on training batch is 42.1779.
After 68001 training step(s), regulayzer on training batch is 3.11244.
Acc on val is 0.890110
Acc on test is 0.865320
After 69001 training step(s), loss on training batch is 25.6235.
After 69001 training step(s), pure_loss on training batch is 44.7222.
After 69001 training step(s), regulayzer on training batch is 3.26239.
Acc on val is 0.879121
Acc on test 

After 99001 training step(s), loss on training batch is 25.3089.
After 99001 training step(s), pure_loss on training batch is 44.6247.
After 99001 training step(s), regulayzer on training batch is 2.99659.
Acc on val is 0.879121
Acc on test is 0.869809
